In [1]:
import os
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from PIL import Image
import cv2
import torch
import torchvision
device = torch.device('cuda')

In [2]:


# Thông số huấn luyện
num_classes = 2  # số nhãn: ví dụ 'none', 'min_30'
batch_size = 16
num_epochs = 10
learning_rate = 1e-4

In [ ]:
# Transform chuẩn ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Tải dữ liệu train & val
train_dataset = datasets.ImageFolder("dataset/train", transform=transform)
val_dataset   = datasets.ImageFolder("dataset/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Lấy tên lớp
categories = train_dataset.classes
print("Các nhãn:", categories)


In [ ]:
# Load mô hình ResNet18
model = models.resnet18(pretrained=True)

# Thay lớp cuối (fc) phù hợp với số nhãn
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)

# Loss và optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)

    # Đánh giá
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"[Epoch {epoch+1}] Loss: {avg_loss:.4f} | Val Acc: {acc:.2f}%")


In [ ]:
torch.save(model.state_dict(), "traffic_sign_model.pth")
print("✅ Đã lưu model vào: traffic_sign_model.pth")


TEST LẠI BẰNG ẢNH

In [ ]:
def predict(image_path):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        pred = torch.argmax(output, 1).item()
        return categories[pred]


img_path = "test_images/min30.jpg"  # Đường dẫn ảnh test
label = predict(img_path)
print("Kết quả dự đoán:", label)
